In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../data/data_transformed.csv")

In [3]:
df.isna().sum()

event-id                        0
location-long                   0
location-lat                    0
argos:lat1                      0
argos:lat2                     85
argos:lon1                      0
argos:lon2                     85
individual-local-identifier     0
date                            0
time                            0
dtype: int64

In [4]:
df = df.dropna()

In [5]:
import datetime as dt

In [6]:
df["year"] = df["date"].apply(lambda x: 
                                    dt.datetime.strptime(x, "%Y-%m-%d").year)


In [7]:
df["month"] = df["date"].apply(lambda x: 
                                    dt.datetime.strptime(x, "%Y-%m-%d").month)

In [8]:
df=df.drop(columns=["event-id", "date", "time", "year", "month"])

In [9]:
for elem in df["individual-local-identifier"].unique():
     df.loc[df["individual-local-identifier"]==elem].to_csv(f"../data/datasets_by_year/{elem}.csv")

In [10]:
df_23043 = pd.read_csv("../data/datasets_by_year/1999CA-Bmu-23043.csv") #individuo con la ruta más larga

In [11]:
df_23043

,Unnamed: 0,location-long,location-lat,argos:lat1,argos:lat2,argos:lon1,argos:lon2,individual-local-identifier
0,3173,-120.143,34.120,34.120,35.544,-120.143,-127.027,1999CA-Bmu-23043
1,3174,-120.145,34.147,34.147,32.243,-120.145,-129.529,1999CA-Bmu-23043
2,3175,-119.740,33.980,33.980,38.156,-119.740,-139.994,1999CA-Bmu-23043
3,3176,-119.771,34.163,34.163,28.573,-119.771,-145.945,1999CA-Bmu-23043
4,3177,-119.688,34.350,34.350,42.993,-119.688,-76.494,1999CA-Bmu-23043
...,...,...,...,...,...,...,...,...
410,3583,-122.007,21.730,21.730,22.057,-122.007,-123.562,1999CA-Bmu-23043
411,3584,-122.871,21.247,21.247,20.439,-122.871,-119.302,1999CA-Bmu-23043
412,3585,-122.755,21.083,21.083,16.252,-122.755,-101.167,1999CA-Bmu-23043
413,3586,-122.272,20.831,20.831,14.861,-122.272,-149.760,1999CA-Bmu-23043


In [12]:
import tensorflow as tf
from tensorflow.data import Dataset

2021-10-20 10:26:25.356136: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-20 10:26:25.356179: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [13]:
df_23043 = df_23043.drop(columns=["Unnamed: 0", "individual-local-identifier"])
#ballena con ruta más larga

In [14]:
df_23043

,location-long,location-lat,argos:lat1,argos:lat2,argos:lon1,argos:lon2
0,-120.143,34.120,34.120,35.544,-120.143,-127.027
1,-120.145,34.147,34.147,32.243,-120.145,-129.529
2,-119.740,33.980,33.980,38.156,-119.740,-139.994
3,-119.771,34.163,34.163,28.573,-119.771,-145.945
4,-119.688,34.350,34.350,42.993,-119.688,-76.494
...,...,...,...,...,...,...
410,-122.007,21.730,21.730,22.057,-122.007,-123.562
411,-122.871,21.247,21.247,20.439,-122.871,-119.302
412,-122.755,21.083,21.083,16.252,-122.755,-101.167
413,-122.272,20.831,20.831,14.861,-122.272,-149.760


In [15]:
df_23043.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415 entries, 0 to 414
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   location-long  415 non-null    float64
 1   location-lat   415 non-null    float64
 2   argos:lat1     415 non-null    float64
 3   argos:lat2     415 non-null    float64
 4   argos:lon1     415 non-null    float64
 5   argos:lon2     415 non-null    float64
dtypes: float64(6)
memory usage: 19.6 KB


In [16]:
n = len(df_23043)
train_df = df_23043[0:int(n*0.7)]
val_df = df_23043[int(n*0.7):int(n*0.9)]
test_df = df_23043[int(n*0.9):]

num_features = df_23043.shape[1]


In [17]:
n_points = 20
window_length = n_points+ 1 #20 puntos que conoce el modelo + 1 que quiero predecir 
train_df = tf.keras.utils.timeseries_dataset_from_array(
    train_df, targets=None,sequence_length = window_length, sequence_stride=1, sampling_rate=1,
    batch_size=32, shuffle=False, seed=None, start_index=None, end_index=None
)

2021-10-20 10:26:33.272853: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-10-20 10:26:33.272897: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-10-20 10:26:33.272914: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-R3AGIN3): /proc/driver/nvidia/version does not exist
2021-10-20 10:26:33.273725: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
n_points = 20
window_length = n_points+ 1 #20 puntos que conoce el modelo + 1 que quiero predecir 
test_df = tf.keras.utils.timeseries_dataset_from_array(
    test_df, targets=None,sequence_length = window_length, sequence_stride=1, sampling_rate=1,
    batch_size=32, shuffle=False, seed=None, start_index=None, end_index=None
)

In [19]:
n_points = 20
window_length = n_points+ 1 #20 puntos que conoce el modelo + 1 que quiero predecir 
val_df = tf.keras.utils.timeseries_dataset_from_array(
    val_df, targets=None,sequence_length = window_length, sequence_stride=1, sampling_rate=1,
    batch_size=32, shuffle=False, seed=None, start_index=None, end_index=None
)

In [20]:
from tensorflow.data import Dataset

In [21]:
def mapdataset(elem):
    X,y = elem[:,:-1,:],elem[:,-1:,0:2]
    X.set_shape([None, n_points, None])
    y.set_shape([None, window_length - n_points, None])
    
    return X,y

In [22]:
train = train_df.map(mapdataset)

In [23]:
test = test_df.map(mapdataset)

In [24]:
val = val_df.map(mapdataset)

In [25]:
lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, return_sequences=True),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=2)
])

In [26]:
MAX_EPOCHS = 300

def compile_and_fit(lstm_model, train, patience=2):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss',
                                                    patience=patience,
                                                    mode='min')
    
    compile = lstm_model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])
    
    
    history = lstm_model.fit(train, epochs=MAX_EPOCHS,
                      validation_data=val,
                      callbacks=[early_stopping])
    return history

In [27]:
history = compile_and_fit(lstm_model, train)

2021-10-20 10:26:38.791968: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/300
9/9 [==============================] - 2s 54ms/step - loss: 7974.0122 - mean_absolute_error: 78.2968 - val_loss: 7658.9473 - val_mean_absolute_error: 76.2025
Epoch 2/300
9/9 [==============================] - 0s 17ms/step - loss: 7911.7964 - mean_absolute_error: 77.9286 - val_loss: 7627.0903 - val_mean_absolute_error: 76.0214
Epoch 3/300
9/9 [==============================] - 0s 17ms/step - loss: 7864.6758 - mean_absolute_error: 77.6776 - val_loss: 7566.2739 - val_mean_absolute_error: 75.6927
Epoch 4/300
9/9 [==============================] - 0s 17ms/step - loss: 7807.7500 - mean_absolute_error: 77.3650 - val_loss: 7501.8569 - val_mean_absolute_error: 75.2902
Epoch 5/300
9/9 [==============================] - 0s 17ms/step - loss: 7713.5059 - mean_absolute_error: 76.7776 - val_loss: 7389.9673 - val_mean_absolute_error: 74.5856
Epoch 6/300
9/9 [==============================] - 0s 17ms/step - loss: 7616.3267 - mean_absolute_error: 76.1669 - val_loss: 7312.8789 - val_mean_abso

Epoch 49/300
9/9 [==============================] - 0s 16ms/step - loss: 6462.1519 - mean_absolute_error: 68.4553 - val_loss: 6223.2339 - val_mean_absolute_error: 66.6020
Epoch 50/300
9/9 [==============================] - 0s 16ms/step - loss: 6442.6763 - mean_absolute_error: 68.3160 - val_loss: 6204.3091 - val_mean_absolute_error: 66.4630
Epoch 51/300
9/9 [==============================] - 0s 16ms/step - loss: 6423.2993 - mean_absolute_error: 68.1773 - val_loss: 6185.4790 - val_mean_absolute_error: 66.3245
Epoch 52/300
9/9 [==============================] - 0s 16ms/step - loss: 6404.0195 - mean_absolute_error: 68.0390 - val_loss: 6166.7437 - val_mean_absolute_error: 66.1865
Epoch 53/300
9/9 [==============================] - 0s 16ms/step - loss: 6384.8320 - mean_absolute_error: 67.9011 - val_loss: 6148.0913 - val_mean_absolute_error: 66.0489
Epoch 54/300
9/9 [==============================] - 0s 16ms/step - loss: 6358.9839 - mean_absolute_error: 67.7256 - val_loss: 6094.1558 - val_mea

Epoch 97/300
9/9 [==============================] - 0s 16ms/step - loss: 5390.2217 - mean_absolute_error: 60.5273 - val_loss: 5180.0327 - val_mean_absolute_error: 58.6720
Epoch 98/300
9/9 [==============================] - 0s 16ms/step - loss: 5371.8320 - mean_absolute_error: 60.3846 - val_loss: 5162.1953 - val_mean_absolute_error: 58.5295
Epoch 99/300
9/9 [==============================] - 0s 20ms/step - loss: 5353.5249 - mean_absolute_error: 60.2422 - val_loss: 5144.4399 - val_mean_absolute_error: 58.3875
Epoch 100/300
9/9 [==============================] - 0s 18ms/step - loss: 5335.3003 - mean_absolute_error: 60.1004 - val_loss: 5126.7637 - val_mean_absolute_error: 58.2458
Epoch 101/300
9/9 [==============================] - 0s 18ms/step - loss: 5317.1475 - mean_absolute_error: 59.9589 - val_loss: 5108.3701 - val_mean_absolute_error: 58.0991
Epoch 102/300
9/9 [==============================] - 0s 17ms/step - loss: 5287.6904 - mean_absolute_error: 59.7401 - val_loss: 5064.5215 - val_

Epoch 145/300
9/9 [==============================] - 0s 16ms/step - loss: 4546.2510 - mean_absolute_error: 53.8039 - val_loss: 4361.5117 - val_mean_absolute_error: 51.9574
Epoch 146/300
9/9 [==============================] - 0s 16ms/step - loss: 4530.7437 - mean_absolute_error: 53.6762 - val_loss: 4346.4883 - val_mean_absolute_error: 51.8299
Epoch 147/300
9/9 [==============================] - 0s 16ms/step - loss: 4515.2905 - mean_absolute_error: 53.5488 - val_loss: 4331.5171 - val_mean_absolute_error: 51.7027
Epoch 148/300
9/9 [==============================] - 0s 16ms/step - loss: 4499.8926 - mean_absolute_error: 53.4217 - val_loss: 4316.5981 - val_mean_absolute_error: 51.5758
Epoch 149/300
9/9 [==============================] - 0s 16ms/step - loss: 4484.5469 - mean_absolute_error: 53.2950 - val_loss: 4301.7319 - val_mean_absolute_error: 51.4493
Epoch 150/300
9/9 [==============================] - 0s 16ms/step - loss: 4469.2544 - mean_absolute_error: 53.1686 - val_loss: 4286.9165 - v

9/9 [==============================] - 0s 16ms/step - loss: 3870.7307 - mean_absolute_error: 48.1311 - val_loss: 3707.3167 - val_mean_absolute_error: 46.2929
Epoch 193/300
9/9 [==============================] - 0s 16ms/step - loss: 3857.4341 - mean_absolute_error: 48.0174 - val_loss: 3694.4456 - val_mean_absolute_error: 46.1794
Epoch 194/300
9/9 [==============================] - 0s 17ms/step - loss: 3844.1782 - mean_absolute_error: 47.9041 - val_loss: 3681.6135 - val_mean_absolute_error: 46.0662
Epoch 195/300
9/9 [==============================] - 0s 16ms/step - loss: 3830.9636 - mean_absolute_error: 47.7910 - val_loss: 3668.8210 - val_mean_absolute_error: 45.9548
Epoch 196/300
9/9 [==============================] - 0s 16ms/step - loss: 3817.7883 - mean_absolute_error: 47.6782 - val_loss: 3656.0681 - val_mean_absolute_error: 45.8457
Epoch 197/300
9/9 [==============================] - 0s 16ms/step - loss: 3804.6541 - mean_absolute_error: 47.5656 - val_loss: 3643.3535 - val_mean_absolu

Epoch 240/300
9/9 [==============================] - 0s 16ms/step - loss: 3275.1619 - mean_absolute_error: 42.9887 - val_loss: 3130.9106 - val_mean_absolute_error: 41.6747
Epoch 241/300
9/9 [==============================] - 0s 15ms/step - loss: 3263.6116 - mean_absolute_error: 42.8882 - val_loss: 3119.7336 - val_mean_absolute_error: 41.5900
Epoch 242/300
9/9 [==============================] - 0s 15ms/step - loss: 3252.0938 - mean_absolute_error: 42.7880 - val_loss: 3108.5881 - val_mean_absolute_error: 41.5055
Epoch 243/300
9/9 [==============================] - 0s 15ms/step - loss: 3240.6084 - mean_absolute_error: 42.6881 - val_loss: 3097.4731 - val_mean_absolute_error: 41.4212
Epoch 244/300
9/9 [==============================] - 0s 16ms/step - loss: 3229.1543 - mean_absolute_error: 42.5884 - val_loss: 3086.3899 - val_mean_absolute_error: 41.3370
Epoch 245/300
9/9 [==============================] - 0s 15ms/step - loss: 3217.7332 - mean_absolute_error: 42.4890 - val_loss: 3075.3374 - v

9/9 [==============================] - 0s 16ms/step - loss: 2765.1545 - mean_absolute_error: 38.5461 - val_loss: 2637.3667 - val_mean_absolute_error: 37.9553
Epoch 288/300
9/9 [==============================] - 0s 16ms/step - loss: 2754.9883 - mean_absolute_error: 38.4576 - val_loss: 2627.5278 - val_mean_absolute_error: 37.8812
Epoch 289/300
9/9 [==============================] - 0s 16ms/step - loss: 2744.8486 - mean_absolute_error: 38.3693 - val_loss: 2617.7151 - val_mean_absolute_error: 37.8072
Epoch 290/300
9/9 [==============================] - 0s 16ms/step - loss: 2734.7356 - mean_absolute_error: 38.2812 - val_loss: 2607.9275 - val_mean_absolute_error: 37.7334
Epoch 291/300
9/9 [==============================] - 0s 16ms/step - loss: 2724.6494 - mean_absolute_error: 38.1934 - val_loss: 2598.1663 - val_mean_absolute_error: 37.6604
Epoch 292/300
9/9 [==============================] - 0s 16ms/step - loss: 2714.5898 - mean_absolute_error: 38.1058 - val_loss: 2588.4302 - val_mean_absolu